In [87]:
pip install nl2query azure-kusto-data 


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [88]:
from nl2query import KustoQuery
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder

In [89]:
cluster_uri = "https://help.kusto.windows.net"

In [90]:
kcsb = KustoConnectionStringBuilder.with_interactive_login(cluster_uri)

In [231]:
with KustoClient(kcsb) as kusto_client:
    database = "ContosoSales"
    query = "Products | getschema"
    response = kusto_client.execute(database, query)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [113]:
with KustoClient(kcsb) as kusto_client:
    database = "FindMyPartner"
    query = "Partner | getschema"
    response = kusto_client.execute(database, query)

    

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [114]:
cols = [x[0] for x in response.primary_results[0]]
print(cols)

['Partner', 'FocusGeography', 'PartnerType', 'Website', 'Contact', 'Logo', 'SuccessStories', 'Description', 'MicrosoftPartnerSite']


In [116]:
queryfier = KustoQuery(cols, 'Partner')


In [81]:
query = queryfier.generate_query('''show me the customers with a first name like "Ang" and a city like "Ball"''')

In [108]:
query = queryfier.generate_query('''show me all manufactures who make color name "Silver"''')

In [124]:
query = queryfier.generate_query('''show me all partner who has contact "info@dataCore.co.il"''')

In [125]:
query

'Partner | where Contact == "info@dataCore.co.il" | project Partner'

In [126]:
with KustoClient(kcsb) as kusto_client:
    database = "FindMyPartner"
    response = kusto_client.execute(database, query)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [123]:
print(response.primary_results[0])

{"name": "PrimaryResult", "kind": "PrimaryResult", "data": []}


In [129]:
with KustoClient(kcsb) as kusto_client:
    database = "Samples"
    query1 = "StormEvents | getschema"
    response1 = kusto_client.execute(database, query1)
    query2 = "PopulationData | getschema"
    response2 = kusto_client.execute(database, query2)

    

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [137]:
cols1 = [x[0] for x in response1.primary_results[0]]
print(cols1)
cols2 = [x[0] for x in response2.primary_results[0]]
print(cols2)
queryfier1 = KustoQuery(cols1, 'StormEvents')
queryfier2 = KustoQuery(cols2, 'PopulationData')

['StartTime', 'EndTime', 'EpisodeId', 'EventId', 'State', 'EventType', 'InjuriesDirect', 'InjuriesIndirect', 'DeathsDirect', 'DeathsIndirect', 'DamageProperty', 'DamageCrops', 'Source', 'BeginLocation', 'EndLocation', 'BeginLat', 'BeginLon', 'EndLat', 'EndLon', 'EpisodeNarrative', 'EventNarrative', 'StormSummary']
['State', 'Population']


In [135]:
query = queryfier.generate_query('''join storemevents table to population data table on state and project eventID and state''')

In [136]:
query

'Populationdata | project State, Population'

In [138]:
with KustoClient(kcsb) as kusto_client:
    database = "ContosoSales"
    query = "SalesFact | getschema"
    response = kusto_client.execute(database, query)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [145]:
cols = [x[0] for x in response.primary_results[0]]
print(cols)
queryfier = KustoQuery(cols, 'SalesFact')

['SalesAmount', 'TotalCost', 'DateKey', 'ProductKey', 'CustomerKey']


In [160]:
query = queryfier.generate_query('''average sales amount  date key "2007-04-03T00:00:00Z " ''')

In [161]:
query

'SalesFact | where DateKey == "2007-04-03T00:00:00Z" | summarize avg(SalesAmount)'

In [168]:
query1 = queryfier.generate_query('''give any top 5 rows of sales fact table ''')

In [169]:
query1

'SalesFact | project SalesAmount | order by SalesAmount desc | limit 5'

In [172]:
query2 = queryfier.generate_query('''give 2 days data before the  date key "2007-04-03T00:00:00Z " ''')

/Users/parthunallapu/Library/Python/3.9/lib/python/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [173]:
query2

'SalesFact | where DateKey < datetime(2007-04-03t00:00:00z) | project TotalCost'

In [ ]:
query2 = queryfier.generate_query('''give 2 days data before the  date key "2007-04-03T00:00:00Z " ''')

In [232]:
query3 = queryfier.generate_query('''average sales amount for product key equal to 2,023 and give top 5 product key  " ''')

In [233]:
query3

'SalesFact | where ProductKey == "2,023" | summarize avg(SalesAmount) by ProductKey | top 5 by avg_SalesAmount desc | project ProductKey'

In [234]:
query4 = queryfier.generate_query('''give me 10 entries of the data''')

In [235]:
query4

'SalesFact | summarize sum(SalesAmount) by TotalCost'

In [198]:
query5 = queryfier.generate_query(''' total number of rows in the table''')

In [199]:
query5

'SalesFact | summarize count(distinct TotalCost)'

In [200]:
query6 = queryfier.generate_query(''' sort the data by sales amount in descending order''')

/Users/parthunallapu/Library/Python/3.9/lib/python/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [201]:
query6

'SalesFact | order by SalesAmount desc'

In [224]:
query7 = queryfier.generate_query('''show top 10 sales amount and total cost of the table''')

In [225]:
query7

'SalesFact | project SalesAmount, TotalCost | order by SalesAmount desc | limit 10'

In [229]:
query8 = queryfier.generate_query('''show me unique product key and sales amount of the table''')

/Users/parthunallapu/Library/Python/3.9/lib/python/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [230]:
query8

'SalesFact | distinct ProductKey, SalesAmount'

In [240]:
query9 = queryfier.generate_query('''show top 10 sales amount   of the table and bottom 10 total cost''')

In [241]:
query9

'SalesFact | project SalesAmount, TotalCost | order by SalesAmount desc | limit 10'